In [1]:
#!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle5 as pickle
import torch

In [2]:
# HIDE OUTPUT
#!wget http://www.atarimania.com/roms/Roms.rar 
#!unrar x -o+ ./Roms.rar >./roms
#!python3 -m atari_py.import_roms ./ROMS >./roms

In [3]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("ALE/MsPacman-v5")

Action Space: Discrete(9)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 27000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [4]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 7
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 3
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 2
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [5]:
n_actions = 9
embed_size = 100800#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
#try:
#    with open(root_dir+f'./genoms/best_genom_pacman.pkl', 'rb') as f:
#        genom = pickle.load(f)
#        genom = genom[-1]
#    print('loaded successfully')
#except Exception:
genom = nt.nn.disassemble_genom()

In [6]:
def test_atary(genom, draw=False,game_name="ALE/MsPacman-v5",seed=0,controller=None):
    env = gym.make(game_name)
    env.reset()
    n_actions = env.action_space.n
    state_dim = env.observation_space.shape#здесь картинка
    #print('n_actions',n_actions,'state_dim',state_dim)
    #1/0
    state_dim = embed_size#это эмбеддинг картинки
    if controller==None: controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
    out_tape = np.zeros(30)
    reward_sum = 0
    #seed=1
    #np.random.seed(seed)
    #env.seed(seed)
    while True:
        action = np.ravel(out_tape)
        action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
        #print('action',int(np.argmax(action)),action)
        #исполнить env
        state, reward, done,_ = env.step(int(np.argmax(action)))
        t=pd.Timestamp.now()
        #state = encode_mobnet(np.array([state]))
        #state = encode_simple(state)
        reward_sum += reward
        #if np.shape(state)[0]>1:
        #    shp = np.shape(state)
        #    state = np.reshape(state,[1,shp[0]])
        if done:
            break
        out_tape = controller.act(torch.tensor(state),reward,done)
        if draw:
            globals()['video'].append(Image.fromarray(env.render("rgb_array")))
    return reward_sum,controller

In [7]:
def multy_test(genom,draw=False):
  if draw:
    globals()['video'] = []

  game_name = "ALE/MsPacman-v5"
  #один энвайронмент, разные сиды
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=100800,output_size=n_actions) 
  q_arr = []
  for i in range(3):
    q,controller = test_atary(genom, draw=draw,game_name=game_name,controller=controller)
    q_arr.append(q*(i+1))
  return np.sum(q_arr)/len(q_arr)#-0.000000001*np.sum(genom**2)

In [8]:
%%time
q,_=test_atary(genom,game_name="ALE/MsPacman-v5", draw=False)
print('q=',q)

q= 160.0
CPU times: user 24.1 s, sys: 136 ms, total: 24.3 s
Wall time: 4.16 s


In [ ]:
%%time
q=multy_test(genom)
print('q=',q)

q= 663.3333333333334
CPU times: user 4min 32s, sys: 250 ms, total: 4min 32s
Wall time: 47.2 s


In [29]:
%%time
q,_=test_atary(genom)
print('q=',q)

q= 90.0
CPU times: user 54.9 s, sys: 24 ms, total: 54.9 s
Wall time: 9.31 s


In [30]:
print(pd.Timestamp.now())

2022-06-05 17:57:35.150790


In [10]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=1,init_file='genoms/genom2.pkl',history_file='history/history2.pkl')

1
history loaded successfully


In [ ]:
try:
    with open('genoms/genom_pacman.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

In [11]:

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open('genoms/best_genom_pacman.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

opt# 0
scores for optimizers augmented [-8.00450000e+01  3.26644167e+02  1.01664167e+03  1.65200000e+00
  2.22666667e+00 -4.20000000e-02 -7.50000000e-02  6.66666667e-02
  1.00000000e+10]
chosen evol_narrow previous_result: nan per tacts: nan
iteration 0 y= [703.33333333 590.        ]
iteration 1 y= [1033.33333333  840.        ]
iteration 2 y= [1013.33333333  820.        ]
iteration 3 y= [750. 550.]
iteration 4 y= [626.66666667 623.33333333]
iteration 5 y= [696.66666667 603.33333333]
iteration 6 y= [690.         683.33333333]
iteration 7 y= [1283.33333333  696.66666667]
iteration 8 y= [623.33333333 570.        ]
iteration 9 y= [706.66666667 673.33333333]
iteration 10 y= [676.66666667 576.66666667]
iteration 11 y= [840.         813.33333333]
iteration 12 y= [810.         563.33333333]
iteration 13 y= [790. 730.]
iteration 14 y= [746.66666667 653.33333333]
iteration 15 y= [866.66666667 706.66666667]
iteration final y= [923.33333333 893.33333333]
result evol_narrow previous_gain: 220.0 per

In [12]:
with open('genoms/genom2.pkl', 'rb') as f:
    genom = pickle.load(f)#[-1]
q=multy_test(genom,draw=True)
print('q=',q)

q= 366.6666666666667


In [13]:
video[0].save(
    './out_videos/seaquest_pacman.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)